<a href="https://colab.research.google.com/github/dajebbar/FreeCodeCamp-python-data-analysis/blob/main/Exercise_M1_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The goal of this exercise is to evaluate the impact of using an arbitrary integer encoding for categorical variables along with a linear classification model such as Logistic Regression.

To do so, let's try to use `OrdinalEncoder` to preprocess the categorical variables. This preprocessor is assembled in a pipeline with `LogisticRegression`. The generalization performance of the pipeline can be evaluated by _cross-validation_ and then compared to the score obtained when using `OneHotEncoder` or to some other baseline score.

First, we load the dataset.

In [1]:
import pandas as pd

adult_census = pd.read_csv("./adult.csv")

In [2]:
target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name, "education-num"])

In [3]:
from sklearn.compose import make_column_selector as selector 

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(data)
data_categorical = data[categorical_columns]

Define a scikit-learn pipeline composed of an OrdinalEncoder and a LogisticRegression classifier.

In [19]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LogisticRegression

model = make_pipeline(OrdinalEncoder(handle_unknown="use_encoded_value", 
                                     unknown_value=-1),
                      LogisticRegression(max_iter=500))

## Evaluate Model

In [20]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model,
                            data_categorical,
                            target,
)

cv_results

{'fit_time': array([0.96557856, 0.69453311, 0.90758133, 0.94574761, 0.7835381 ]),
 'score_time': array([0.07087135, 0.07288694, 0.07205296, 0.07378292, 0.06938338]),
 'test_score': array([0.75504146, 0.75555328, 0.75573301, 0.75307125, 0.75788288])}

In [21]:
scores = cv_results['test_score']

f"{scores.mean():.3f} '+/-'  {scores.std():.3f}"

"0.755 '+/-'  0.002"

In [14]:
from sklearn.preprocessing import OneHotEncoder
model2 = make_pipeline(OneHotEncoder(handle_unknown='ignore'),
                       LogisticRegression(max_iter=500))

In [17]:
cv_results = cross_validate(model2,
                            data_categorical,
                            target,
)

cv_results

{'fit_time': array([1.88570929, 1.37368798, 1.5548985 , 1.35697341, 1.48296356]),
 'score_time': array([0.03757548, 0.03818607, 0.03807187, 0.05203223, 0.0398953 ]),
 'test_score': array([0.83222438, 0.83560242, 0.82872645, 0.83312858, 0.83466421])}

In [18]:
scores = cv_results['test_score']

f"{scores.mean():.3f} '+/-'  {scores.std():.3f}"

"0.833 '+/-'  0.002"